# 01. ICU Cohort Extraction
## Base ICU Cohort: BigQuery Build + Sanity Checks

---

## 0. Overview
This notebook defines a standardized ICU cohort in BigQuery, which serves as the
common starting point for all downstream cohort construction and survival analyses.

---

## 1. Purpose
The purpose of this notebook is to construct a reusable ICU-level cohort from
MIMIC-IV, providing a consistent foundation for disease-specific cohorts and
exposure definitions in subsequent notebooks (02–04).

---

## 2. Data Sources
- **MIMIC-IV v3.1** (BigQuery public dataset)
- Tables:
  - `icu.icustays`
  - `hosp.admissions`
  - `hosp.patients`

---

## 3. Common BigQuery Utilities
This section initializes the BigQuery client using Application Default Credentials (ADC)
and defines shared helper functions for executing SQL scripts and retrieving query results.
These utilities are reused across all downstream notebooks.

- `run_sql_script(path)`: executes DDL / CREATE TABLE SQL scripts
- `query_to_df(query)`: runs SELECT queries and returns a pandas DataFrame

In [ ]:
# Use Application Default Credentials (my user account)
# This account already has PhysioNet BigQuery access.

from google.cloud import bigquery
from google.auth import default
from pathlib import Path

# Define project ID
PROJECT_ID = "mimic-iv-portfolio"

# Get ADC credentials
creds, adc_project = default()

# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID, credentials=creds)

print("Connected to BigQuery project:", PROJECT_ID)
print("ADC default project:", adc_project)

# Helper to run a SQL script file (DDL, CREATE TABLE, etc.)
def run_sql_script(path) :
    """
    Read a .sql file from disk, execute it in BigQuery,
    and wait until the job finishes.
    Use this for CREATE TABLE / INSERT INTO scripts.
    """
    sql_path = Path(path)
    with sql_path.open("r") as f:
        query = f.read()
    job = client.query(query)
    job.result()
    print(f"Executed SQL script: {sql_path.name}")

# Helper for SELECT queries → DataFrame
def query_to_df(query) :
    """
    Run a SELECT query in BigQuery and return a pandas DataFrame.
    """
    job = client.query(query)
    return job.to_dataframe(create_bqstorage_client=False)

Connected to BigQuery project: mimic-iv-portfolio
ADC default project: mimic-iv-portfolio


---

## 4. Build Cohort Tables in BigQuery
This step executes SQL scripts that define the standardized ICU cohort table
directly in BigQuery. The cohort is constructed using ICU stay identifiers
(`subject_id`, `hadm_id`, `stay_id`) and temporal information (`intime`, `outtime`).

In [2]:
# Excecute SQL script (01_extract_base.sql)
run_sql_script("../sql/01_extract_base.sql")

Executed SQL script: 01_extract_base.sql


**ICU Cohort Construction (01_extract_base_icu.sql)**

This SQL script (01_extract_base_icu.sql) constructs the standardized adult ICU
cohort `mimic-iv-portfolio.copd_raas.cohort_icu`, which serves as the foundation
for all downstream cohort and exposure construction steps (02–04).

The cohort is derived from the MIMIC-IV ICU stays table
(`physionet-data.mimiciv_3_1_icu.icustays`) and enriched with patient-level
demographics and age information from the hospital patients table
(`physionet-data.mimiciv_3_1_hosp.patients`).

Specifically, the query:
- Computes ICU length of stay at the ICU-stay level
- Restricts the cohort to adult patients (age ≥ 18)
- Retains only the first ICU stay per hospital admission to ensure independent observations

The resulting table provides a clean and reproducible ICU-stay–level cohort, used as the common input for disease-specific cohort construction and all subsequent survival analyses.

---

## 5. Sanity Checks
Basic checks are performed to confirm:
- Expected cohort size
- Presence of unique ICU stay identifiers
- Valid ICU admission and discharge times

In [3]:
# Sanity checks for the created base ICU cohort table
df_preview = query_to_df("""
    SELECT *
    FROM `mimic-iv-portfolio.copd_raas.cohort_icu`
""")
print("\nPreview of cohort_icu:")
df_preview.info()


Preview of cohort_icu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85242 entries, 0 to 85241
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   subject_id         85242 non-null  Int64         
 1   hadm_id            85242 non-null  Int64         
 2   stay_id            85242 non-null  Int64         
 3   intime             85242 non-null  datetime64[us]
 4   outtime            85228 non-null  datetime64[us]
 5   icu_los            85228 non-null  float64       
 6   age                85242 non-null  Int64         
 7   gender             85242 non-null  object        
 8   anchor_year        85242 non-null  Int64         
 9   anchor_year_group  85242 non-null  object        
dtypes: Int64(5), datetime64[us](2), float64(1), object(2)
memory usage: 6.9+ MB


---

## 6. Outputs and Downstream Use
The finalized ICU cohort table is saved as:

- **Table:** `copd_raas.cohort_icu`
- **Write mode:** `CREATE OR REPLACE`

This table serves as the common input for all subsequent cohort construction,
exposure definition, and survival analysis notebooks (02–04).